In [ ]:
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import joblib
import FinanceDataReader as fdr
import numpy as np

model = load_model('full_kospi_model.h5')

In [ ]:
stock_code = input("종목코드 : ex) 066570 :")

stock_data = fdr.DataReader(stock_code)

last_day_data = stock_data.iloc[-1]

num_data_points = 50 # 10일간의 데이터로 학습해서 10일 추천
# 많은 일수를 넣을수록 정확하다고 생각함
# 급격한 변동이 있을땐 일수를 늘려보는것도 좋다
# num_data_points = int(input("일수 관여:"))

model_input = []
for i in range(num_data_points):
    day_data = stock_data.iloc[-(num_data_points - i)]
    model_input.append([day_data['Close'], day_data['Volume']])

model_input = np.array([model_input])
predicted_price = model.predict(model_input)

current_price = last_day_data['Close']
pred_price = current_price * predicted_price

if predicted_price > 1:
    print("주식 가격은 상승할 것으로 예측됩니다.")
else:
    print("주식 가격은 하락할 것으로 예측됩니다.")
print(f"현 가격 : {int(current_price)}원")
print(f"예측 값 : {int(pred_price)}원")

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

font_path = "C:/Windows/Fonts/malgun.ttf".
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

plt.figure(figsize=(12, 6))

# 실제 주가 데이터 그래프
plt.plot(stock_data.index, stock_data['Close'], label='주가', color='blue')

# 예측된 주가 표시
next_date = stock_data.index[-1] + pd.DateOffset(1)
plt.scatter(next_date, pred_price, color='red', label='예측된 다음날 종가')
plt.text(next_date, pred_price, f'{int(pred_price)}원', fontsize=10, va='bottom', ha='left', color='red')

# 현재 가격 표시
plt.scatter(stock_data.index[-1], stock_data['Close'][-1], color='green', label='마지막 종가')
plt.text(stock_data.index[-1], stock_data['Close'][-1], f'{int(stock_data["Close"][-1])}원', fontsize=10, va='bottom', ha='left', color='green')

plt.title('주가 예측')
plt.xlabel('날짜')
plt.ylabel('가격')
plt.legend()
plt.grid(True)
plt.show()
